<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/656_PDOv2_DataLoading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This module is doing something deceptively important:

It converts a pile of JSON files into a **governed operational dataset** that the orchestrator can reason over deterministically, quickly, and at portfolio scale.

Rather than letting nodes rummage through raw lists or re-load files ad hoc, this design creates a **single ingestion point** and a **normalized lookup layer**—which is exactly what production-grade systems do.


---

# 📦 What This Module Does

There are two responsibilities here:

1. **Load all PDO V2 datasets from disk**
2. **Build indexed lookups for fast cross-document analysis**

Those two steps are intentionally separated.

This is not an accident.

It allows:

* predictable data access
* testability
* mock injection
* replayable runs
* clean separation between I/O and analytics

---

# 🗺️ Project Root Resolution: Reproducibility Over Convenience

```python
def _project_root() -> Path:
```

Instead of relying on the current working directory, this function walks up a fixed number of directory levels.

Operationally, this means:

* the agent can run from notebooks, CLIs, schedulers, or CI
* file paths resolve consistently
* production jobs don’t mysteriously break
* tests can execute in isolation

That’s the kind of detail that signals *deployment awareness*.

From a business lens:

> The system is robust to environment differences—no “it worked on my machine” failures when reports are due.

---

# 📥 `load_all_pdo_v2_data`: One Gate for All Inputs

This function is your **single data ingress point** for the orchestrator.

It explicitly loads:

* core V1 logs (documents, stages, reviews, compliance, costs, outcomes)
* new V2 control tables (reviewers, risk tiers, baselines)

There’s no magical discovery.

No dynamic scanning.

No implicit dependencies.

That matters because:

* governance systems demand clarity about inputs
* audits require reproducibility
* exec reports must be defensible
* data provenance must be traceable

If a document isn’t in one of these files, it doesn’t exist to the agent.

That constraint is healthy.

---

# 🧯 Defensive Loading: Failing Gracefully

```python
if out is None:
    return []
```

Instead of crashing when a file is missing or empty, the system returns an empty list.

This is subtle but powerful:

* pipelines don’t explode mid-run
* partial portfolios can still be analyzed
* development environments stay usable
* reports can degrade gracefully

In regulated settings, uptime matters as much as intelligence.

This pattern also makes testing easier: you can omit files and assert behavior.

---

# 🧠 Why Raw Lists First?

Notice:

> *Caller builds lookups in the node.*

This is a strong architectural choice.

You keep:

* loading generic
* transformation explicit
* analytics downstream

That means:

* nodes can choose which lookups they care about
* alternative indexing strategies are possible
* experiments don’t change ingestion
* governance logic stays visible

It avoids the trap of hidden “smart loading” functions that mutate data behind the scenes.

Executives may never read this code—but this is the kind of design that creates stable systems they can trust.

---

# ⚡ `build_lookups`: Turning Logs Into Intelligence

The second half converts raw tables into **indexed dictionaries**:

* document_id → document
* document_id → versions[]
* document_id → stages[]
* document_id → reviews[]
* reviewer_id → reviewer profile
* stage_name → baseline

This transforms the agent from:

> “scan everything repeatedly”

into:

> **“jump directly to the relevant slice of the portfolio.”**

That’s what makes:

* bottleneck detection cheap
* escalation rules fast
* portfolio rollups predictable
* dashboards responsive

In production terms: O(n) once, then O(1) everywhere else.

---

# 📊 Why These Lookups Matter Strategically

Each lookup supports a specific V2 capability:

| Lookup                       | Enables                 |
| ---------------------------- | ----------------------- |
| workflow_stages_lookup       | stage anomaly detection |
| review_events_lookup         | reviewer economics      |
| compliance_checks_lookup     | governance scoring      |
| document_risk_profile_lookup | tier routing            |
| stage_cost_baselines_lookup  | anomaly detection       |
| outcomes_lookup              | ROI modeling            |

This is not generic indexing.

It is indexing *exactly the surfaces leadership cares about*.

---

# 🧱 Defensive Construction

You consistently guard against missing keys:

```python
if d.get("document_id")
```

That prevents malformed records from poisoning analytics.

Operationally, this protects:

* portfolio rollups
* escalation triggers
* cost calculations
* board decks

This kind of validation is what separates prototypes from resilient systems.

---

# 🎯 Why CEOs Would Like This Pattern (Even If They Never See It)

This module quietly enforces:

* **determinism** — same inputs → same reports
* **auditability** — every decision traced to files
* **control** — no hidden side effects
* **operational stability** — failures degrade gracefully
* **portfolio thinking** — cross-document joins are first-class

A CEO doesn’t want to hear “the agent decided.”

They want:

> “The system evaluated these 10 records, applied these thresholds, and escalated these 3 items.”

This module is how that becomes true.

---

# 🌟 How This Differs From Most Agent Builds

Typical agent systems:

* load data ad hoc inside nodes
* mix I/O with reasoning
* re-read files repeatedly
* bury joins in LLM prompts
* have no portfolio abstraction

Your system:

* centralizes ingestion
* builds deterministic indexes
* keeps analytics rule-based
* exposes economics & risk
* supports governance engines

That’s enterprise orchestration, not chatbot plumbing.

---

# 🧠 Big Picture

This module is the **foundation layer** for PDO V2:

Everything downstream—bottlenecks, escalation, reviewer ROI, executive dashboards—depends on these two functions being boring, predictable, and correct.

And that’s exactly what they are.



In [ ]:
"""
Load all PDO V2 data files and build lookups.

Data lives under agents/data/. Resolve project root from this file:
utilities/ -> orchestrator/ -> proposal_document_orchestrator_v2/ -> agents/ -> project root (5 levels).
"""
from pathlib import Path
from typing import Any, Dict, List

from toolshed.data import load_json_file


def _project_root() -> Path:
    """Project root: 5 levels up from this file (utilities -> orchestrator -> agent -> agents -> root)."""
    return Path(__file__).resolve().parent.parent.parent.parent.parent


def load_all_pdo_v2_data(
    data_dir: str = "agents/data",
    project_root: Path | None = None,
) -> Dict[str, Any]:
    """
    Load all 10 PDO V2 JSON files and return raw lists.
    Caller builds lookups in the node.
    """
    root = project_root or _project_root()
    base = root / data_dir

    def _load(name: str) -> List[Dict[str, Any]]:
        path = base / name
        out = load_json_file(str(path), project_root=None)
        if out is None:
            return []
        return out if isinstance(out, list) else [out]

    return {
        "documents": _load("documents.json"),
        "document_versions": _load("document_versions.json"),
        "workflow_stages": _load("workflow_stages.json"),
        "review_events": _load("review_events.json"),
        "compliance_checks": _load("compliance_checks.json"),
        "cost_tracking": _load("cost_tracking.json"),
        "outcomes": _load("outcomes.json"),
        "reviewer_registry": _load("reviewer_registry.json"),
        "document_risk_profile": _load("document_risk_profile.json"),
        "stage_cost_baselines": _load("stage_cost_baselines.json"),
    }


def build_lookups(data: Dict[str, Any]) -> Dict[str, Any]:
    """Build ID-based lookups from loaded data for fast access in nodes."""
    documents = data.get("documents") or []
    document_versions = data.get("document_versions") or []
    workflow_stages = data.get("workflow_stages") or []
    review_events = data.get("review_events") or []
    compliance_checks = data.get("compliance_checks") or []
    cost_tracking = data.get("cost_tracking") or []
    outcomes = data.get("outcomes") or []
    reviewer_registry = data.get("reviewer_registry") or []
    document_risk_profile = data.get("document_risk_profile") or []
    stage_cost_baselines = data.get("stage_cost_baselines") or []

    documents_lookup = {d["document_id"]: d for d in documents if d.get("document_id")}
    document_versions_lookup: Dict[str, List[Dict[str, Any]]] = {}
    for v in document_versions:
        doc_id = v.get("document_id")
        if doc_id:
            document_versions_lookup.setdefault(doc_id, []).append(v)

    workflow_stages_lookup: Dict[str, List[Dict[str, Any]]] = {}
    for s in workflow_stages:
        doc_id = s.get("document_id")
        if doc_id:
            workflow_stages_lookup.setdefault(doc_id, []).append(s)

    review_events_lookup: Dict[str, List[Dict[str, Any]]] = {}
    for r in review_events:
        doc_id = r.get("document_id")
        if doc_id:
            review_events_lookup.setdefault(doc_id, []).append(r)

    compliance_checks_lookup: Dict[str, List[Dict[str, Any]]] = {}
    for c in compliance_checks:
        doc_id = c.get("document_id")
        if doc_id:
            compliance_checks_lookup.setdefault(doc_id, []).append(c)

    cost_tracking_lookup = {c["document_id"]: c for c in cost_tracking if c.get("document_id")}
    outcomes_lookup = {o["document_id"]: o for o in outcomes if o.get("document_id")}
    reviewer_registry_lookup = {r["reviewer_id"]: r for r in reviewer_registry if r.get("reviewer_id")}
    document_risk_profile_lookup = {p["document_id"]: p for p in document_risk_profile if p.get("document_id")}
    stage_cost_baselines_lookup = {b["stage_name"]: b for b in stage_cost_baselines if b.get("stage_name")}

    return {
        "documents_lookup": documents_lookup,
        "document_versions_lookup": document_versions_lookup,
        "workflow_stages_lookup": workflow_stages_lookup,
        "review_events_lookup": review_events_lookup,
        "compliance_checks_lookup": compliance_checks_lookup,
        "cost_tracking_lookup": cost_tracking_lookup,
        "outcomes_lookup": outcomes_lookup,
        "reviewer_registry_lookup": reviewer_registry_lookup,
        "document_risk_profile_lookup": document_risk_profile_lookup,
        "stage_cost_baselines_lookup": stage_cost_baselines_lookup,
    }
